### 最近在项目中有很多用到dataframe处理搜集的数据场景，这里记录一些使用dataframe的总结

In [3]:
import pandas as pd
import numpy as np
from IPython.display import display

### dataframe的创建

In [4]:
# 直接使用DataFrame 创建
pd.DataFrame([[1,1], [2,2], [3,3], [4,4]], columns=list('AB'))
# 从csv文件中读取
pd.read_csv('./cluster.csv')

# 从ndarray构建
pd.DataFrame(np.random.randn(3,4), columns=list('ABCD'))

# 从Series中构建
d = pd.Series([1,2,3,4,5,6], dtype='float')
pd.DataFrame(d)

# 从Series中可以创建一些有意思的类型，比如日期等
#d = pd.date_range('20130101', periods=6)
d = pd.Series(['2013-01-01', '2013-01-02', '2013-01-03'], \
          dtype='datetime64[ns]')
pd.DataFrame(np.random.randn(3,3), index=d)

# 分别构建dataframe中的每一列，来构建最终的dataframe
# DataFrame中的每一列应该是一个series(todo)
df1 = pd.DataFrame({ 'A' : 1.,
                    'B' : pd.Timestamp('20130102'),
                    'C' : pd.Series(1,index=list(range(4)),dtype='float32')})
display (df1)

,A,B,C
0,1.0,2013-01-02,1.0
1,1.0,2013-01-02,1.0
2,1.0,2013-01-02,1.0
3,1.0,2013-01-02,1.0


### dataframe过滤，筛选满足条件的记录

In [5]:
ft = pd.read_csv('./cluster.csv')
display (ft.head(5))

,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicatessen,cluster
0,3,12669,9656,7561,214,2674,1338,1
1,3,7057,9810,9568,1762,3293,1776,1
2,3,6353,8808,7684,2405,3516,7844,1
3,3,13265,1196,4221,6404,507,1788,0
4,3,22615,5410,7198,3915,1777,5185,1


In [6]:
display (ft[(ft['cluster'] == 1) & (ft['Fresh'] > 10000)]. head(5))

,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicatessen,cluster
0,3,12669,9656,7561,214,2674,1338,1
4,3,22615,5410,7198,3915,1777,5185,1
6,3,12126,3199,6975,480,3140,545,1
12,3,31714,12319,11757,287,3881,2931,1
13,3,21217,6208,14982,3095,6707,602,1


In [7]:
# (ft['cluster'] == 1).shape print (type(ft['cluster'] == 1))
# 输出由bool值构成的Series 然后dataframe根据这个Series来做过滤。

In [8]:
t = pd.DataFrame(ft['Fresh'].apply(lambda x: str(x)))
t[t['Fresh'].str.contains('1266')]
# 我们需要按照条件输出一个bool数组来告诉DataFrame哪些记录需要保存下来
# 使用哪些方法可以由我们自己来选择。

,Fresh
0,12669


In [9]:
### 使用Group BY 来进行分类统计

In [10]:

dfg = pd.DataFrame({'col_1': pd.Series(range(1, 7)),
                    'col_2': pd.Series(range(1, 7) *10),
              'type' : pd.Series(['t1','t1','t1',
                                't2','t2','t2'])})
#print (dfg.index)

In [11]:
# 统计每种type的个数
dfgb = dfg.groupby(['type', 'col_2']).count()
print (dfgb.index)
display (dfgb)

MultiIndex(levels=[[u't1', u't2'], [1, 2, 3, 4, 5, 6]],
           labels=[[0, 0, 0, 1, 1, 1], [0, 1, 2, 3, 4, 5]],
           names=[u'type', u'col_2'])


col_1
type col_2       
t1   1          1
     2          1
     3          1
t2   4          1
     5          1
     6          1

### 多表之间的链接操作

In [31]:
# 使用 concat缺省值会被自动不上NaN
df1 = pd.DataFrame([[1,2,3], [4,5,6], [1,2,3]], columns=list('ABC'))
df2 = pd.DataFrame([[7,8,9], [10,11,12], [1,4,4]], columns=list("ABD"))

display(pd.concat([df1,df2]))

,A,B,C,D
0,1,2,3.0,NaN
1,4,5,6.0,NaN
2,1,2,3.0,NaN
0,7,8,NaN,9.0
1,10,11,NaN,12.0
2,1,4,NaN,4.0


### 使用left inner right 分别做表连接
- inner链接相当于去两个表,满足条件的交集，不满足条件的行则会被去掉。
- left 链接除了取两个表的交集，还会保留left表中没有匹配上的行记录。
- right 和left相反会保留right表中没有匹配上的行记录

In [32]:
display(pd.merge(left=df1, right=df2, on= ['A'], how='inner'))

,A,B_x,C,B_y,D
0,1,2,3,4,4
1,1,2,3,4,4


In [33]:
display(pd.merge(left=df1, right=df2, on= ['A'], how='left'))

,A,B_x,C,B_y,D
0,1,2,3,4.0,4.0
1,4,5,6,NaN,NaN
2,1,2,3,4.0,4.0


In [34]:
display(pd.merge(left=df1, right=df2, on= ['A'], how='right'))

,A,B_x,C,B_y,D
0,1,2.0,3.0,4,4
1,1,2.0,3.0,4,4
2,7,NaN,NaN,8,9
3,10,NaN,NaN,11,12
